# TO-DO STEPS PISANE NA PIERWSZYCH ZAJĘCIACH

Data Scraping:


## Data Processing (dla obu zbiorów danych)
1. Ułożenie danych w kolumnach.
2. Miejsca zerowe.
3. Feature Engineering. (kolumna koordynaty geograficzne, 


## Data analysis. (+potencjalnie Streamlit)
1. Prócz metrażu czynniki najbardziej wpływające na cenę.
2. Analiza ceny do piętra
3. Analiza odległości geograficznej od centrum
4. Analiza parking/balon
5. Analiza ceny/trendów w konkretnych dzielniach.


# Jeśli model to.
5. Onehotencoding.
6. Train/Test
7. Model.

Bonus points:
    wizualizacja geograficzna (streamlit - prettymap/mapbox)
    stosunek ilości balkonów w mieszkaniu na dzielnice (balkon rate)
    wpływ i częstotliwość '!' w nazwie oferty i ich zestawienie z ceną
    

# TO-DO 16.04.2024

1. Docelowo proponuję, żeby pierwszy etap (pisany w jednym notatniku jupiter) zamknął się po data preparation i data engeenering, po tym etapie zapiszemy dane do plików do dalszych analiz (wielkrotne pobieranie koordynatów może być bolesne). Aby to osiągnąć trzeba:
  - Uspójnić notatniki (na razie jeszcze nie dane :P), tak, byśmy pracowali na jednym.
  - Dodać na początek chaina uzupełnienie zbiorów o nowe dane (można wyciąć fragmenty z utils) - tam, gdzie będziemy analizować dane statycznie i/lub przestrzennie (pierwotny plan).
  - Jeśli idzie o analize w szeregach czasowych, to trochę inne para kaloszy, można zostawić na później -- chyba, że ktorś będzie chciał się tym zająć.
  - Uważam, że trzeba wybrać, które analizy przeprowadzamy na uspójnionych z obu zbiorów danych, a które osobno na zbiorach - otodom ma więcej cech, nieruchomości online (chyba) bardziej rozróżnia pewne cechy (np. te garaże) - Do decyzji.
  - Z obu zbiorów usunąć "duplikaty" - różne ogłoszenia tego samego mieszkania. Tu trzeba będzie przyjąć jakąś heurystykę.
  - Uzupełnić koordynaty
  - Na ich podstawie dodać kolumnę z odległością od wyróżnionego miejsca (centrum) w metrach.
  - Zbiory zapisać


  - Jeśli idzie o szeregi czasowe, to IMO data i cena w badanych punktach czasowych starczy (cokolwiek innego się tam jeszcze mogło zmieniać). Będzie trzeba wybrać tylko te oferty, które zmieniały się w tej kolumnie.

2. Etap analiz:
- Sprawdzić, co z labów Czecha jest do wykorzystania, bo mam przeczycie, że coś z tego, co u nie go robiliśmy się przyda - PCA chociażby i ten (dual) scatterplot.
- Z drugich labów Czecha ten pearson corelation, to macierz korelacji, którą i tak planowaliśmy.
- Na SISI będziemy trenować model drzew decyzyjnych, więc można go tu dodać też.
- Trzeba przejrzeć, co jeszcze planowaliśmy i co jest w "literaturze" (bo znalzałem coś takiego), co jeszcze można przeprowadzić.


3. Wizualizacja:
Poza oczywstymi wykresikami, tabelami itp, ta heatmapa (oraz heatmapa+, jak ze stramlita) wydaje mi się fajnym bajerem. Można spróbować się podjąć. W oógle wydaje mi się, że w tym aspekcie geograficznym można będzie fajnie nie tylko przeprowadzać analizy, ale też pokazywać rzeczy.

Ważna rzecz to dokumentowanie prac (albo w readme, albo zacząć odrębny dokument) nawet, gdy nic nie wychodzi.


To są oczywiście moje wstępne i ogólne pomysły i jeśli macie inne, to śmiało wędrujcie w te strony, które Wam się wydadzą sensowne i słuszne.

    

# Zbiór Otodom

## Załadowanie i agregacja danych

In [1]:
import numpy as np
import pandas as pd

In [3]:
df_otodom = pd.read_csv('datasets/timestamps/2024-03-08-otodom_dataset_raw.csv')
df_2024_03_25_otodom_new_data = pd.read_csv('datasets/timestamps/2024-03-25-otodom-new_data.csv')
df_2024_03_25_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-03-25-otodom-updated_data.csv')
df_2024_04_07_otodom_new_data = pd.read_csv('datasets/timestamps/2024-04-07-otodom-new_data.csv')
df_2024_04_07_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-04-07-otodom-updated_data.csv')
df_2024_04_21_otodom_new_data = pd.read_csv('datasets/timestamps/2024-04-21-otodom-new_data.csv')
df_2024_04_21_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-04-21-otodom-updated_data.csv')
df_2024_05_05_otodom_new_data = pd.read_csv('datasets/timestamps/2024-05-05-otodom_new_data.csv')
df_2024_05_05_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-05-05-otodom_updated_data.csv')

In [4]:
def update_dataframe(oldest_df, updated_df, new_df):
    def set_url_as_index(df):
        if 'url' in df.columns:
            df.set_index('url', inplace=True)
        else:
            raise ValueError("URL column not found. Please check the column names.")
    try:
        set_url_as_index(oldest_df)
        set_url_as_index(updated_df)
        set_url_as_index(new_df)
    except ValueError as e:
        print(e)
        return None
    
    oldest_df.update(updated_df)
    merged_df = pd.concat([oldest_df, new_df], axis=0)
    merged_df.reset_index(inplace=True)

    return merged_df

In [5]:
df_otodom = update_dataframe(df_otodom, df_2024_03_25_otodom_updated_data, df_2024_03_25_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_04_07_otodom_updated_data, df_2024_04_07_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_04_21_otodom_updated_data, df_2024_04_21_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_05_05_otodom_updated_data, df_2024_05_05_otodom_new_data)

In [6]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14172 entries, 0 to 14171
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   url                   14172 non-null  object
 1   name/title            14172 non-null  object
 2   address               14172 non-null  object
 3   price                 14172 non-null  object
 4   area                  14172 non-null  object
 5   price-per-area        13492 non-null  object
 6   floor/store           13943 non-null  object
 7   no of rooms           14171 non-null  object
 8   year of construction  12627 non-null  object
 9   parking space         8531 non-null   object
 10  market                12627 non-null  object
 11  form of ownership     10867 non-null  object
 12  condition             11335 non-null  object
 13  rent                  6636 non-null   object
 14  heating               10426 non-null  object
 15  advertiser type       12627 non-null

## Przygotowanie danych

### Cena

#### Usunięcie brakujących danych, uspójnienie oraz zmiana na typy liczbowe

In [7]:
# Wierszy bez ceny
no_price = df_otodom[~df_otodom['price'].str.contains('\d')]
no_price['price'].count()

687

In [8]:
# Usunięcie wierszy z brakującą ceną (no_price)
df_otodom.drop(no_price.index, inplace=True)
df_otodom['price'].count()

13485

#### Ceny podane w innej walucie

In [9]:
# Wiersze w EUR
different_currency = df_otodom[~df_otodom['price'].str.contains('zł', na=False)].copy()
different_currency['price'].count()

11

In [10]:
different_currency['price-per-area'] = different_currency['price-per-area'].str.replace(' ', '').str.extract(r'(\d+\.\d+|\d+)').astype(float)
different_currency['area'] = different_currency['area'].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)').astype(float)

In [11]:
df_otodom.loc[different_currency.index]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
1265,https://www.otodom.pl/pl/oferta/penthouse-232m...,Penthouse 232m w zabytkowej kamienicy Stare Mi...,"Nowy Świat, Stare Miasto, Kraków, małopolskie",1 149 999 EUR,"232,1 m²",21 451 zł/m²,4/4,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,nie,NaN,kamienica,cegła
2056,https://www.otodom.pl/pl/oferta/penthouse-w-po...,Penthouse w pobliżu Rynku Głównego,"marszałka JózefaPiłsudskiego, Nowy Świat, Star...",1 150 000 EUR,232 m²,21 460 zł/m²,4/5,9,1906,NaN,wtórny,pełna własność,do zamieszkania,1 700 zł,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
3144,https://www.otodom.pl/pl/oferta/5-pokoi-taras-...,5 pokoi | Taras 77m2 | Komórka | Centrum | Mar...,"ul. marsz. Józefa Piłsudskiego, Nowy Świat, St...",1 149 999 EUR,"232,1 m²",21 451 zł/m²,4/4,5,1908,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,850 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",585 280 EUR,"91,45 m²",27 708 zł/m²,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4722,https://www.otodom.pl/pl/oferta/penthouse-tara...,Penthouse tarasem na dachu w kamienicy z windą,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"228,7 m²",21 770 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,NaN,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 896 zł/m²,4/5,5.0,1906,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,220 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
4995,https://www.otodom.pl/pl/oferta/penthouse-star...,Penthouse Stare Miasto II Poziomy. Taras 70m2,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"232,1 m²",21 451 zł/m²,4/5,5,1906,NaN,wtórny,NaN,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji
9359,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 837 zł/m²,4/5,5,1906,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,220 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
13044,https://www.otodom.pl/pl/oferta/mieszkanie-swi...,Mieszkanie świetna lokalizacja Kraków-Śródmieście,"ulica Grochowska 27 B / 1B, Osiedle Oficerskie...",255 000 EUR,"107,3 m²",10 289 zł/m²,2/2,3,1981,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,NaN,gazowe,prywatny,nie,"balkon, ogródek",szeregowiec,cegła
13565,https://www.otodom.pl/pl/oferta/krakowska-kami...,Krakowska kamienica,"ul. Mazowiecka, Krowodrza, Krowodrza, Kraków, ...",380 000 EUR,84 m²,19 585 zł/m²,5/6,3,1936,NaN,wtórny,pełna własność,do zamieszkania,880 zł,miejskie,prywatny,tak,balkon,kamienica,cegła


In [12]:
# zamiana cen na wartości liczbowe
df_otodom['price'] = df_otodom['price'].str.replace(' ', '').str.replace(',', '.')
df_otodom['price'] = df_otodom['price'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [13]:
# przeliczanie ceny na podstawie metrażu i ceny za metr podanej w PLN
different_currency.loc[:, 'price'] = (different_currency['price-per-area'] * different_currency['area'])
df_otodom.loc[different_currency.index, 'price'] = different_currency['price'].astype(float)
df_otodom.loc[different_currency.index, 'price']

1265     4978777.1
2056     4978720.0
3144     4978777.1
3977     2533896.6
4722     4978799.0
4908     4992288.0
4995     4978777.1
9359     4978836.0
13044    1104009.7
13565    1645140.0
13596    2714565.0
Name: price, dtype: float64

In [14]:
# Usunięcie złapanego wiersza z mieszkaniem z Chorwacji
df_otodom.drop(index=3977, inplace=True)

In [15]:
df_otodom['price'].count()

13484

### Zamiana wartości w kolumnach na typy liczbowe

#### metraż

In [16]:
df_otodom['area'] = df_otodom['area'].str.replace(' m²', '')
df_otodom['area'] = df_otodom['area'].str.replace(' ', '')
df_otodom['area'] = df_otodom['area'].str.replace(',', '.').astype(float)

#### cena za metr

In [17]:
# df_otodom[df_otodom['price-per-area'].str.contains(r'\d', na=False).shape[0] & ~df_otodom['price-per-area'].str.contains('zł', na=False)].shape[0]
df_otodom[(df_otodom['price-per-area'].str.contains(r'\d', na=False)) & (~df_otodom['price-per-area'].str.contains('zł', na=False))].shape[0]

0

In [18]:
df_otodom[df_otodom['price-per-area'].isnull()].shape[0]

0

In [19]:
df_otodom['price-per-area'] = df_otodom['price-per-area'].str.replace(' zł/m²', '').str.replace(' ', '').str.replace(',', '.')
df_otodom['price-per-area'] = df_otodom['price-per-area'].astype(float)

#### czynsz

In [20]:
# Wiersze w innej walucie
df_otodom[df_otodom['rent'].str.contains(r'\d', na=False) & ~df_otodom['rent'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
6335,https://www.otodom.pl/pl/oferta/2m-wysoki-stan...,2M! Wysoki Standard| Gotowe| Balkon | Widok!,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",620000.0,35.55,17440.0,7/11,2,2019,NaN,wtórny,pełna własność,do zamieszkania,420 GBP,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji


In [21]:
eur_rent_price = df_otodom[df_otodom['rent'].str.contains('EUR', na=False)]
gbp_rent_price = df_otodom[df_otodom['rent'].str.contains('GBP', na=False)]

In [22]:
# zamiana wartości rent na typ liczbowy
df_otodom['rent'] = df_otodom['rent'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

In [23]:
import yfinance as yf

eur = yf.Ticker("EURPLN=X")
gbp = yf.Ticker("GBPPLN=X")
euro_exchange_rate = eur.info['fiftyDayAverage']
gbp_exchange_rate = gbp.info['fiftyDayAverage']
print(euro_exchange_rate)
print(gbp_exchange_rate)

4.3039
5.029518


In [24]:
# euro_exchange_rate = 4.28
# gbp_exchange_rate = 4.99

df_otodom.loc[eur_rent_price.index, 'rent'] *= euro_exchange_rate
df_otodom.loc[gbp_rent_price.index, 'rent'] *= gbp_exchange_rate

In [25]:
# Średnia po wycięciu outliersów do ew. uzupełnienia
mean_rent = df_otodom['rent'].mean()
std_rent = df_otodom['rent'].std()

filtered_std_rent = df_otodom[(df_otodom['rent'] > (mean_rent - std_rent)) & 
                        (df_otodom['rent'] < (mean_rent + std_rent))]

new_mean_rent = filtered_std_rent['rent'].mean()
new_mean_rent

542.4641154168552

#### liczba pokoi

In [26]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.replace(' ', '')
print(df_otodom[df_otodom['no of rooms'].isna()].shape[0])
df_otodom.dropna(subset=['no of rooms'], inplace=True)

334


In [27]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)

#### piętro

In [28]:
# Rozdzielenie kolumny 'floor/store' na dwie osobne kolumny
df_otodom[['floor/store', 'no of floors/stores in the building']] = df_otodom['floor/store'].str.split('/', expand=True)

In [29]:
# Usunięcie wierszy bez informacji o piętrze
print(df_otodom[df_otodom['floor/store'].isnull()].shape[0])
df_otodom = df_otodom.dropna(subset=['floor/store'])

226


In [30]:
# Zamiana wartości "parter" na 0 w kolumnie "floor"
df_otodom['floor/store'] = df_otodom['floor/store'].replace('parter', 0)

In [31]:
# Zamiana wartości "suterena" na -1 w kolumnie "floor"
df_otodom['floor/store'] = df_otodom['floor/store'].replace('suterena', -1)

In [32]:
# Konwersja kolumny "floor" na typ liczbowy, zamieniając nieliczbowe wartości na NaN
df_otodom['floor/store'] = pd.to_numeric(df_otodom['floor/store'], errors='coerce')

# Usunięcie wierszy zawierających wartości NaN w kolumnie "floor"
df_otodom = df_otodom[~df_otodom['floor/store'].isna()]

In [33]:
df_otodom['floor/store'] = df_otodom['floor/store'].astype(int)

#### liczba pięter

In [34]:
print(df_otodom[df_otodom['no of floors/stores in the building'].isnull()].shape[0])
df_otodom = df_otodom.dropna(subset=['no of floors/stores in the building'])

488


In [35]:
df_otodom['no of floors/stores in the building'] = df_otodom['no of floors/stores in the building'].astype(int)

In [36]:
#Reorder
columns = list(df_otodom.columns)
columns.remove('no of floors/stores in the building')
target_index = columns.index('floor/store')
columns.insert(target_index + 1, 'no of floors/stores in the building')
df_otodom = df_otodom[columns]

In [37]:
# Według wikipedii najwyższy mieszkalny budunek w Krk ma 19 pięter
# https://pl.wikipedia.org/wiki/Lista_najwyższych_budynków_w_Krakowie
print(df_otodom[df_otodom['no of floors/stores in the building'] > 19].shape[0])
df_otodom.drop(df_otodom[df_otodom['no of floors/stores in the building'] > 19].index, inplace=True)

1


#### rok budowy

In [38]:
df_otodom['year of construction'] = pd.to_numeric(df_otodom['year of construction'], errors='coerce')

missing_below_1200 = df_otodom[df_otodom['year of construction'] < 1250]
missing_above_2026 = df_otodom[df_otodom['year of construction'] > 2026]
pd.concat([missing_above_2026, missing_below_1200])

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
187,https://www.otodom.pl/pl/oferta/4-pokoje-nowe-...,"4 pokoje, nowe, Promocja -175 tys. taniej","Stare Podgórze, Podgórze, Kraków, małopolskie",998358.0,62.10,16077.0,1,4,4,2924.0,...,pierwotny,pełna własność,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,balkon,apartamentowiec,brak informacji
7888,https://www.otodom.pl/pl/oferta/mieszkanie-na-...,"Mieszkanie na sprzedaż, Kraków 40,2m2","ul. Janusza Supniewskiego, Osiedle Oficerskie,...",699000.0,40.20,17388.0,0,2,2,20100.0,...,wtórny,pełna własność,do zamieszkania,350.0,miejskie,prywatny,nie,NaN,blok,cegła
10812,https://www.otodom.pl/pl/oferta/widokowe-3-pok...,"Widokowe 3 pok, 2 loggie, garaż podziemny, Ban...","ul. Stefana Banacha, Górka Narodowa, Prądnik B...",759000.0,49.47,15343.0,6,8,3,2924.0,...,pierwotny,pełna własność,do wykończenia,1.0,miejskie,biuro nieruchomości,tak,NaN,blok,brak informacji
6685,https://www.otodom.pl/pl/oferta/nowe-3-pokoje-...,Nowe: 3 pokoje na Ruczaju. Garaż podziemny.,"Ruczaj, Dębniki, Kraków, małopolskie",790300.0,56.45,14000.0,3,3,3,1024.0,...,pierwotny,pełna własność,do wykończenia,250.0,miejskie,biuro nieruchomości,tak,balkon,blok,pustak
11292,https://www.otodom.pl/pl/oferta/3-pokoje-na-bi...,3 Pokoje na Bieńczycach! Wysoki Standard!,"os. Wysokie, Bieńczyce, Bieńczyce, Kraków, mał...",669000.0,45.00,14867.0,3,4,3,450.0,...,wtórny,pełna własność,do zamieszkania,450.0,miejskie,biuro nieruchomości,nie,NaN,blok,cegła
11328,https://www.otodom.pl/pl/oferta/mieszkanie-na-...,Mieszkanie na sprzedaż- Mogilska-76 m2 + garaż,"ul. Mogilska, Osiedle Oficerskie, Grzegórzki, ...",1080000.0,76.00,14211.0,8,10,3,87.0,...,wtórny,pełna własność,do zamieszkania,1200.0,miejskie,biuro nieruchomości,tak,balkon,blok,wielka płyta
12986,https://www.otodom.pl/pl/oferta/nowy-apartamen...,Nowy apartament z ogródkiem 90 m2. Bez Prowizji.,"ul. Pana Tadeusza, Zabłocie, Podgórze, Kraków,...",794151.0,39.15,20285.0,0,7,2,204.0,...,pierwotny,NaN,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,"balkon, ogródek",brak informacji,brak informacji


In [39]:
# Poprawianie "z placa"
df_otodom['year of construction'] = df_otodom['year of construction'].replace(2924.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(1024.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(204.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(20100.0, 2010)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(87.0, 1987)
df_otodom.drop(df_otodom[df_otodom['year of construction'] == 450.0].index, inplace=True)

In [40]:
# df_otodom['year of construction'] = df_otodom['year of construction'].fillna(-1).astype(int)
df_otodom['year of construction'].value_counts(dropna=False).head()

year of construction
NaN       2373
2024.0    1934
2023.0    1637
2025.0     553
2022.0     381
Name: count, dtype: int64

In [41]:
# Średnia po wycięciu outliersów do ew. uzupełnienia
mean_year = df_otodom['year of construction'].mean()
std_year = df_otodom['year of construction'].std()

filtered_df = df_otodom[(df_otodom['year of construction'] > (mean_year - std_year)) & 
                        (df_otodom['year of construction'] < (mean_year + std_year))]

new_mean_year = filtered_df['year of construction'].mean()
new_mean_year = int(new_mean_year)
new_mean_year

2012

#### rynek

In [42]:
df_otodom.loc[(df_otodom['market'].isna()) & (df_otodom['year of construction'] > 2022), 'market'] = 'pierwotny'
df_otodom.loc[(df_otodom['market'].isna()) & (df_otodom['year of construction'] < 2023), 'market'] = 'wtórny'

In [43]:
print(df_otodom['market'].isna().sum())
df_otodom.dropna(subset=['market'], inplace=True)

931


In [44]:
df_otodom['market'].value_counts(dropna=False)

market
wtórny       7445
pierwotny    3952
Name: count, dtype: int64

#### rodzaj własności

In [45]:
df_otodom['form of ownership'].value_counts(dropna=False)

form of ownership
pełna własność                       7990
NaN                                  2999
spółdzielcze wł. prawo do lokalu      293
udział                                111
użytkowanie wieczyste / dzierżawa       4
Name: count, dtype: int64

In [46]:
# Zgodnie z obowiązującym prawem 2007 r. nie ma możliwości ustanawiania spółdzielczego własnościowego prawa do lokalu
df_otodom.loc[(df_otodom['form of ownership'].isna()) & (df_otodom['year of construction'] > 2006), 'form of ownership'] = 'pełna własność'

#### stan

In [47]:
df_otodom['condition'].value_counts(dropna=False)

condition
do zamieszkania    4889
do wykończenia     3448
NaN                2550
do remontu          510
Name: count, dtype: int64

#### parking

In [48]:
df_otodom['parking space'].value_counts(dropna=False)

parking space
garaż/miejsce parkingowe    6360
NaN                         5037
Name: count, dtype: int64

In [49]:
df_otodom['parking space'] = df_otodom['parking space'].notna()
df_otodom['parking space'].value_counts(dropna=False)

parking space
True     6360
False    5037
Name: count, dtype: int64

#### Ogrzewanie

In [50]:
df_otodom['heating'].value_counts(dropna=False)

heating
miejskie         7215
NaN              2125
gazowe           1297
inne              395
elektryczne       212
kotłownia         151
piece kaflowe       2
Name: count, dtype: int64

#### balkon

In [51]:
df_otodom['outdoor area'].value_counts(dropna=False)

outdoor area
balkon                    6337
NaN                       2797
ogródek                    570
balkon, taras              497
balkon, ogródek, taras     369
taras                      290
ogródek, taras             282
balkon, ogródek            201
taras, balkon               34
taras, ogródek              17
taras, balkon, ogródek       3
Name: count, dtype: int64

In [52]:
df_otodom['outdoor area'] = df_otodom['outdoor area'].fillna('None')

def standardize_outdoor_area(area):
    if area == 'None':
        return area
    parts = sorted(set(area.replace(' ', '').split(',')))
    return ', '.join(parts)

df_otodom['outdoor area'] = df_otodom['outdoor area'].apply(standardize_outdoor_area)
# outdoor_encoded = df_otodom['outdoor area'].str.get_dummies(sep=', ')
# df_otodom = df_otodom.join(outdoor_encoded)

#### ogłoszeniodawca

In [53]:
df_otodom['advertiser type'].value_counts(dropna=False)

advertiser type
biuro nieruchomości    9935
prywatny               1306
deweloper               156
Name: count, dtype: int64

In [54]:
df_otodom.loc[(df_otodom['advertiser type'].isna()), 'advertiser type'] = 'biuro nieruchomości'

#### winda

In [55]:
df_otodom['elevator'].value_counts(dropna=False)

elevator
tak    6133
nie    5264
Name: count, dtype: int64

In [56]:
# Zgodnie z obowiązującym od 1960 prawem budowalnym
df_otodom.loc[df_otodom['elevator'].isna() & (df_otodom['no of floors/stores in the building'] > 4), 'elevator'] = 'tak'
df_otodom['elevator'] = df_otodom['elevator'].map({'tak': True, 'nie': False, np.nan: False})
df_otodom['elevator'] = df_otodom['elevator'].astype(bool)

#### typ budynku

In [57]:
df_otodom['building type'].value_counts(dropna=False)

building type
blok                5779
apartamentowiec     2222
brak informacji     1931
kamienica           1147
dom wolnostojący     208
szeregowiec           97
plomba                10
loft                   3
Name: count, dtype: int64

#### materiał

In [58]:
df_otodom['building type'].value_counts(dropna=False)

building type
blok                5779
apartamentowiec     2222
brak informacji     1931
kamienica           1147
dom wolnostojący     208
szeregowiec           97
plomba                10
loft                   3
Name: count, dtype: int64

## Adresy - wstępnie

In [59]:
# Usuń adresy spoza Krakowa
print(df_otodom[~df_otodom['address'].str.contains("Kraków", na=False)].shape[0])
df_otodom = df_otodom[df_otodom['address'].str.contains("Kraków", na=False)]

136


In [60]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11261 entries, 0 to 14171
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  11261 non-null  object 
 1   name/title                           11261 non-null  object 
 2   address                              11261 non-null  object 
 3   price                                11261 non-null  float64
 4   area                                 11261 non-null  float64
 5   price-per-area                       11261 non-null  float64
 6   floor/store                          11261 non-null  int32  
 7   no of floors/stores in the building  11261 non-null  int32  
 8   no of rooms                          11261 non-null  int32  
 9   year of construction                 9823 non-null   float64
 10  parking space                        11261 non-null  bool   
 11  market                           

In [61]:
df_otodom.isnull().sum()

url                                       0
name/title                                0
address                                   0
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building       0
no of rooms                               0
year of construction                   1438
parking space                             0
market                                    0
form of ownership                      1113
condition                              2541
rent                                   5233
heating                                2097
advertiser type                           0
elevator                                  0
outdoor area                              0
building type                             0
building material                         0
dtype: int64

In [62]:
nan_row_count = df_otodom.isna().any(axis=1).sum()
df_otodom.shape[0] - nan_row_count

4133

# Zbiór danych nieruchomosci-online

### Załadowanie i agregacja danych

In [64]:
df_nieruchomosci = pd.read_csv('datasets/timestamps/2024-03-08-nieruchomosci-online_dataset_raw.csv')
df_2024_03_25_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-03-25-nieruchomosci-online-new_data.csv')
df_2024_03_25_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-03-25-nieruchomosci-online-updated_data.csv')
df_2024_04_07_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-04-07-nieruchomosci-online-new_data.csv')
df_2024_04_07_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-04-07-nieruchomosci-online-updated_data.csv')
df_2024_04_21_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-04-21-nieruchomosci-online-new_data.csv')
df_2024_04_21_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-04-21-nieruchomosci-online-updated_data.csv')
df_2024_05_05_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-05-05-nieruchomosci-online_new_data.csv')
df_2024_05_05_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-05-05-nieruchomosci-online_updated_data.csv')

In [65]:
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_03_25_nieruchomosci_updated_data, df_2024_03_25_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_04_07_nieruchomosci_updated_data, df_2024_04_07_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_04_21_nieruchomosci_updated_data, df_2024_04_21_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_05_05_nieruchomosci_updated_data, df_2024_05_05_nieruchomosci_new_data)

In [66]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  9659 non-null   object 
 1   name/title                           9659 non-null   object 
 2   address                              9290 non-null   object 
 3   price                                9536 non-null   object 
 4   area                                 9659 non-null   object 
 5   price-per-area                       9536 non-null   object 
 6   floor/store                          9659 non-null   object 
 7   no of floors/stores in the building  9027 non-null   float64
 8   no of rooms                          9659 non-null   object 
 9   year of construction                 9659 non-null   object 
 10  parking space                        9659 non-null   object 
 11  market                        

### Cena i cena za metr
#### Usunięcie wierszy z brakującymi danymi

In [67]:
print(df_nieruchomosci[df_nieruchomosci['price'].isnull()].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['price'].isnull()].index, inplace=True)

123


In [68]:
# sprawdzenie, czy są jakieś wartości w kolumnie price-per-area, które nie zawierają danych liczbowych
df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains(r'\d')].shape[0]

0

In [69]:
# sprawdzenie, czy są jakieś wartości w kolumnie price, które nie zawierają danych liczbowych
df_nieruchomosci[~df_nieruchomosci['price'].str.contains(r'\d')].shape[0]

0

### Zamiana cen podanych w innych walutach

In [70]:
# wartości w kolumnie 'price' podane w innej walucie
# price-per-area w innej walucie

# df_nieruchomosci[~df_nieruchomosci['price'].str.contains('zł')].copy()
# df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains('zł')].copy()

different_currency_price = pd.concat([df_nieruchomosci[~df_nieruchomosci['price'].str.contains('zł')].copy(), df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains('zł')].copy()]).drop_duplicates()
different_currency_price

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
153,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1906,parking strzeżony w pobliżu,wtórny,NaN
1120,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1908,garaż,wtórny,NaN
1151,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,"232,10 m²","4 954,76 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,NaN
2096,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,228 m²,"5 043,86 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,własność
2241,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Nowy Świat, Kraków, małopolskie",1 150 000 €,"228,70 m²","5 028,42 €/m²",4,5.0,5,1906,-,wtórny,NaN
2562,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Kraków, małopolskie",138 000 €,65 m²,"2 123,08 €/m²",21,45.0,2,2023,-,wtórny,NaN
2927,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, Plac Juliusza Kossaka","Plac Juliusza Kossaka, Stare Miasto, Kraków, m...",700 000 €,152 m²,"4 605,26 €/m²",1,4.0,4,1930,-,wtórny,NaN
2991,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"marszałka JózefaPiłsudskiego, Wawel, Kraków, m...",1 150 000 €,232 m²,"4 956,90 €/m²",4,5.0,9,1906,-,wtórny,NaN
3364,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie Stare Miasto, ul. Jana","Jana, Stare Miasto, Kraków, małopolskie",1 000 000 €,"221,19 m²",4 521 €/m²,4,4.0,4,1850,-,wtórny,NaN
8480,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Mazowiecka","Mazowiecka, Krowodrza, Kraków, małopolskie",370 000 €,84 m²,"4 404,76 €/m²",5,6.0,3,1936,-,wtórny,NaN


In [71]:
# zamiana cen na wartości liczbowe
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

In [72]:
print(df_nieruchomosci.loc[different_currency_price.index, ['price', 'price-per-area']])

          price  price-per-area
153   1149999.0         4954.76
1120  1149999.0         4954.76
1151  1150000.0         4954.76
2096  1150000.0         5043.86
2241  1150000.0         5028.42
2562   138000.0         2123.08
2927   700000.0         4605.26
2991  1150000.0         4956.90
3364  1000000.0         4521.00
8480   370000.0         4404.76


In [73]:
# kurs na 5.04.2024
# euro_exchange_rate = 4.28

df_nieruchomosci.loc[different_currency_price.index, 'price'] *= euro_exchange_rate
df_nieruchomosci.loc[different_currency_price.index, 'price-per-area'] *= euro_exchange_rate

In [74]:
df_nieruchomosci.loc[different_currency_price.index, ['price', 'price-per-area']]

,price,price-per-area
153,4.949481e+06,21324.791564
1120,4.949481e+06,21324.791564
1151,4.949485e+06,21324.791564
2096,4.949485e+06,21708.269054
2241,4.949485e+06,21641.816838
2562,5.939382e+05,9137.524012
2927,3.012730e+06,19820.578514
2991,4.949485e+06,21334.001910
3364,4.303900e+06,19457.931900
8480,1.592443e+06,18957.646564


### Usunięcie wierszy z przesunięciem danych w kolumnach

In [75]:

print(df_nieruchomosci['no of rooms'].str.contains('-', na=False).sum())
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of rooms'].str.contains('-', na=False)].index, inplace=True)


18


In [76]:
print(df_nieruchomosci['no of rooms'].str.contains(r'\D', na=False).sum())
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of rooms'].str.contains(r'\D', na=False)].index, inplace=True)


67


### Liczba pokoji, i pięter

#### Zamiana wartości w kolumnach na typ liczbowy

In [77]:
df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].astype(int)

In [78]:
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(' m²', '').str.replace(',', '.').astype(float)

In [79]:
df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains('parter')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1200000.0,71.06,16887.14,parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",1198680.0,71.35,16800.00,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",599000.0,40.00,14975.00,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta"
12,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Ułanów","Ułanów, Prądnik Czerwony, Kraków, małopolskie",790000.0,58.00,13620.69,parter,10.0,3,1962,-,wtórny,"własność, księga wieczysta"
18,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Obozowa","Obozowa, Ruczaj, Kraków, małopolskie",689900.0,51.94,13282.63,parter,7.0,2,2012,przynależne na ulicy,wtórny,"własność, księga wieczysta"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9637,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Reja","Reja 9, Żabiniec, Kraków, małopolskie",1100000.0,73.00,15068.49,parter,6.0,3,2005,-,NaN,NaN
9640,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Kobierzyńska","Kobierzyńska, Dębniki, Kraków, małopolskie",899000.0,48.46,18551.38,parter,6.0,2,2022,tak,wtórny,NaN
9644,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Królowej Jadwigi","Królowej Jadwigi, Wola Justowska, Kraków, mało...",894000.0,51.97,17202.00,parter,1.0,3,2023,w garażu podziemnym,pierwotny (zobacz inne nowe mieszkania w Krako...,NaN
9646,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Ułanów","Ułanów, Prądnik Czerwony, Kraków, małopolskie",790000.0,58.00,13620.69,parter,10.0,3,1962,-,wtórny,NaN


In [80]:
# Zamiana wartości "parter" na 0 w kolumnie "floor/store"
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('parter', '0')

In [81]:
# Zamiana wartości "suterena" na -1 w kolumnie "floor/store"
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('suterena', '-1')

In [82]:
# usunięcie wierszy, które w kolumnie 'floor/store' zawierają dane nieliczbowe
print(df_nieruchomosci[~df_nieruchomosci['floor/store'].str.contains(r'\d', na=False)].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[~df_nieruchomosci['floor/store'].str.contains(r'\d')].index, inplace=True)

108


In [83]:
# przesunięte dane
print(df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains(r'[a-zA-Z]')].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains(r'[a-zA-Z]')].index, inplace=True)

147


In [84]:
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].astype(int)

In [85]:
print(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'].isna()].shape[0])
df_nieruchomosci.dropna(subset=['no of floors/stores in the building'], inplace=True)
df_nieruchomosci['no of floors/stores in the building'] = df_nieruchomosci['no of floors/stores in the building'].astype(int)

185


In [86]:
# Według wikipedii najwyższy mieszkalny budunek w Krk ma 19 pięter
# https://pl.wikipedia.org/wiki/Lista_najwyższych_budynków_w_Krakowie
print(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'] > 19].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'] > 19].index, inplace=True)

1


### Rok budowy

In [87]:
df_nieruchomosci['year of construction'] = pd.to_numeric(df_nieruchomosci['year of construction'], errors='coerce')

missing_below_1200 = df_nieruchomosci[df_nieruchomosci['year of construction'] < 1250]
missing_above_2026 = df_nieruchomosci[df_nieruchomosci['year of construction'] > 2026]
pd.concat([missing_above_2026, missing_below_1200])

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
7393,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Banacha","Banacha, Prądnik Biały, Kraków, małopolskie",759000.0,49.47,15342.63,6,8,3,2924.0,tak,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
2100,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Magnolii","Magnolii, Ruczaj, Kraków, małopolskie",790300.0,56.45,14000.00,3,3,3,1024.0,garaż,pierwotny (zobacz inne nowe mieszkania w Krako...,NaN


In [88]:
# Poprawianie "z placa"
df_nieruchomosci['year of construction'] = df_nieruchomosci['year of construction'].replace(2924.0, 2024)
df_nieruchomosci['year of construction'] = df_nieruchomosci['year of construction'].replace(1024.0, 2024)

In [89]:
df_nieruchomosci['year of construction'].value_counts(dropna=False)

year of construction
2024.0    1559
2023.0    1511
NaN        789
2025.0     703
2022.0     399
          ... 
1878.0       1
1921.0       1
1946.0       1
1350.0       1
1845.0       1
Name: count, Length: 153, dtype: int64

### Parking

In [90]:
df_nieruchomosci['parking space'] = df_nieruchomosci['parking space'].replace('-', 'parking publiczny / na ulicy')

In [91]:
df_nieruchomosci['parking space'].unique()

array(['tak', 'w garażu podziemnym', 'parking publiczny / na ulicy',
       'garaż w bryle budynku', 'garaż', 'przynależne na ulicy',
       'przynależne na terenie ogrodzonym', 'możliwość wykupienia',
       'garaż wolnostojący', 'parking strzeżony w pobliżu',
       'wiata garażowa'], dtype=object)

In [92]:
df_nieruchomosci['parking space'].value_counts(dropna=False)

parking space
parking publiczny / na ulicy         3477
w garażu podziemnym                  1602
tak                                  1125
garaż                                 855
przynależne na ulicy                  658
garaż wolnostojący                    471
przynależne na terenie ogrodzonym     243
garaż w bryle budynku                 218
parking strzeżony w pobliżu           195
możliwość wykupienia                  138
wiata garażowa                         28
Name: count, dtype: int64

In [93]:
# Skopiowanie danych do nowej kolumny (mogą się przydać do jakiegoś modelu) i konwersja na boolean dla spójności z otodom
df_nieruchomosci['parking space details'] = df_nieruchomosci['parking space']
false_values = ['parking publiczny / na ulicy', 'możliwość wykupienia', 'parking strzeżony w pobliżu']
df_nieruchomosci['parking space'] = ~df_nieruchomosci['parking space'].isin(false_values)

### Rynek

In [94]:
valid_values = ['wtórny', 'pierwotny (zobacz inne nowe mieszkania w Krakowie)']

shifted_rows = ~df_nieruchomosci['market'].isin(valid_values) & df_nieruchomosci['market'].notna()

df_nieruchomosci.drop(df_nieruchomosci[shifted_rows].index, inplace=True)

In [95]:
df_nieruchomosci.loc[(df_nieruchomosci['market'].isna()) & (df_nieruchomosci['year of construction'] > 2022), 'market'] = 'pierwotny'
df_nieruchomosci.loc[(df_nieruchomosci['market'].isna()) & (df_nieruchomosci['year of construction'] < 2023), 'market'] = 'wtórny'
print(df_nieruchomosci['market'].isna().sum())
df_nieruchomosci.dropna(subset=['market'], inplace=True)

37


In [96]:
df_nieruchomosci['market'] = df_nieruchomosci['market'].replace('pierwotny (zobacz inne nowe mieszkania w Krakowie)', 'pierwotny')

### Forma własności

In [97]:
# Zgodnie z obowiązującym prawem od 2007 r. nie ma możliwości ustanawiania spółdzielczego własnościowego prawa do lokalu
df_nieruchomosci.loc[(df_nieruchomosci['form of ownership'].isna()) & (df_nieruchomosci['year of construction'] > 2006), 'form of ownership'] = 'własność'

In [98]:
ownership_mapping = {
    'własność': 'pełna własność',
    'własność, księga wieczysta': 'pełna własność',
    'spółdzielcze własnościowe, księga wieczysta': 'spółdzielcze wł. prawo do lokalu',
    'spółdzielcze własnościowe': 'spółdzielcze wł. prawo do lokalu',
    'udział w KW': 'udział',
    'udział ze wskazaniem, KW na budynku': 'udział',
    'księga wieczysta': 'pełna własność',
    'inna': 'użytkowanie wieczyste / dzierżawa',
    'udział': 'udział',
    'inna, księga wieczysta': 'użytkowanie wieczyste / dzierżawa',
    'Udział ze wskazaniem': 'udział',
    'Udział': 'udział',
    'udział, księga wieczysta': 'udział',
    'udział ze wskazaniem': 'udział',
    'Chce dopłacić': np.nan,
    'tak': np.nan,
    'udziały, księga wieczysta': 'udział',
    'Oczekuje dopłaty': np.nan,
    'KW budynek, udział ze wskazaniem': 'udział',
    'Spółdzielcza wł. z KW': 'spółdzielcze wł. prawo do lokalu'
}

df_nieruchomosci['form of ownership'] = df_nieruchomosci['form of ownership'].replace(ownership_mapping)

# df_nieruchomosci['form of ownership'].fillna('NaN', inplace=True)


In [99]:
df_nieruchomosci['form of ownership'].value_counts(dropna=False)

form of ownership
pełna własność                       7026
NaN                                  1663
spółdzielcze wł. prawo do lokalu      228
użytkowanie wieczyste / dzierżawa      34
udział                                 13
Name: count, dtype: int64

In [100]:
nan_row_count = df_nieruchomosci.isna().any(axis=1).sum()
df_nieruchomosci.shape[0] - nan_row_count

7129

In [101]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8964 entries, 0 to 9658
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  8964 non-null   object 
 1   name/title                           8964 non-null   object 
 2   address                              8964 non-null   object 
 3   price                                8964 non-null   float64
 4   area                                 8964 non-null   float64
 5   price-per-area                       8964 non-null   float64
 6   floor/store                          8964 non-null   int32  
 7   no of floors/stores in the building  8964 non-null   int32  
 8   no of rooms                          8964 non-null   int32  
 9   year of construction                 8214 non-null   float64
 10  parking space                        8964 non-null   bool   
 11  market                             

In [102]:
df_nieruchomosci.describe()

,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction
count,8.964000e+03,8964.00000,8964.000000,8964.000000,8964.000000,8964.000000,8214.000000
mean,9.746706e+05,58.46407,16852.452356,2.382753,4.863900,2.641678,2002.943146
std,6.413338e+05,30.58719,4848.414384,2.177389,2.449808,1.297148,37.876427
min,1.490000e+05,11.70000,19.140000,-1.000000,1.000000,1.000000,1300.000000
25%,6.600000e+05,40.50000,14000.000000,1.000000,3.000000,2.000000,1997.000000
50%,8.150000e+05,52.00000,16176.470000,2.000000,4.000000,3.000000,2022.000000
75%,1.065000e+06,67.00000,18779.340000,4.000000,6.000000,3.000000,2024.000000
max,1.600000e+07,627.00000,152594.090000,15.000000,17.000000,44.000000,2026.000000


In [103]:
df_nieruchomosci.isnull().sum()

url                                       0
name/title                                0
address                                   0
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building       0
no of rooms                               0
year of construction                    750
parking space                             0
market                                    0
form of ownership                      1663
parking space details                     0
dtype: int64

## Adresy - wstępnie

In [104]:
df_nieruchomosci[~df_nieruchomosci['address'].str.contains("Kraków", na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details


# Inżynieria danych

## Uspójnianie zbiorów

In [105]:
common_columns = df_otodom.columns.intersection(df_nieruchomosci.columns)
df_apartments = pd.concat([df_otodom[common_columns], df_nieruchomosci[common_columns]], ignore_index=True)
df_apartments

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://www.otodom.pl/pl/oferta/gotowe-2-pokoj...,Gotowe| 2 pokoje| blisko centrum| Bonarka,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",719000.0,55.98,12844.00,0,8,3,2022.0,False,pierwotny,pełna własność
1,https://www.otodom.pl/pl/oferta/4-pok-mieszkan...,4-pok.mieszkanie z Sauną - Wysoki Standard ! 2...,"Biskupa Albina Małysiaka, Kobierzyn, Dębniki, ...",1350000.0,87.00,15517.00,2,4,4,2021.0,True,wtórny,pełna własność
2,https://www.otodom.pl/pl/oferta/3-pokoje-w-rza...,3 pokoje w rządowym programie kredyt 2%,"Bieżanów, Bieżanów-Prokocim, Kraków, małopolskie",599000.0,47.10,12718.00,1,4,3,2023.0,True,pierwotny,pełna własność
3,https://www.otodom.pl/pl/oferta/mieszkanie-ide...,"Mieszkanie idealne na start, 3 - pokoje!","ul. Erazma Jerzmanowskiego, Prokocim, Bieżanów...",833500.0,66.69,12498.00,3,8,3,2024.0,True,pierwotny,pełna własność
4,https://www.otodom.pl/pl/oferta/ul-lasowka-3-p...,"ul. Lasówka, 3 pokoje, 65m2 + taras 20m2!","ul. Lasówka, Płaszów, Podgórze, Kraków, małopo...",949000.0,65.81,14420.00,7,7,3,2023.0,False,wtórny,pełna własność
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Kobierzyńska","Kobierzyńska, Mateczny, Kraków, małopolskie",659000.0,35.50,18563.38,4,4,2,2024.0,True,pierwotny,pełna własność
20221,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Retoryka","Retoryka, Stare Miasto, Kraków, małopolskie",1750000.0,95.00,18421.05,2,4,3,1938.0,False,wtórny,pełna własność
20222,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Bieżanowska","Bieżanowska, Bieżanów, Kraków, małopolskie",659000.0,50.00,13180.00,9,11,3,1975.0,True,wtórny,NaN
20223,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Lubostroń","Lubostroń, Ruczaj, Kraków, małopolskie",715000.0,41.00,17439.02,1,3,2,2008.0,False,wtórny,pełna własność


In [106]:
df_apartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20225 entries, 0 to 20224
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  20225 non-null  object 
 1   name/title                           20225 non-null  object 
 2   address                              20225 non-null  object 
 3   price                                20225 non-null  float64
 4   area                                 20225 non-null  float64
 5   price-per-area                       20225 non-null  float64
 6   floor/store                          20225 non-null  int32  
 7   no of floors/stores in the building  20225 non-null  int32  
 8   no of rooms                          20225 non-null  int32  
 9   year of construction                 18037 non-null  float64
 10  parking space                        20225 non-null  bool   
 11  market                      

## Duplikaty

In [107]:
def check_similarity(group):
    if len(group) > 1:
        price_mean = group['price'].mean()
        area_mean = group['area'].mean()
        price_range = price_mean * 0.01
        area_range = area_mean * 0.01
        similar_price = group['price'].between(price_mean - price_range, price_mean + price_range)
        similar_area = group['area'].between(area_mean - area_range, area_mean + area_range)
        return (similar_price & similar_area).rename('is_duplicate')
    else:
        return pd.Series(False, index=group.index, name='is_duplicate')

def fill_from_group(group):
    group['non_null_count'] = group.notna().sum(axis=1)
    sorted_group = group.sort_values('non_null_count', ascending=False)
    sorted_group.drop('non_null_count', axis=1, inplace=True)
    most_complete_row = sorted_group.iloc[0]
    for _, row in sorted_group.iloc[1:].iterrows():
        most_complete_row = most_complete_row.combine_first(row)
    return most_complete_row

def remove_duplicates(df, group_cols=None):
    if group_cols is None:
        group_cols = ['address', 'floor/store', 'no of floors/stores in the building', 'no of rooms']
    
    df['is_duplicate'] = df.groupby(group_cols, group_keys=False).apply(check_similarity)
    
    duplicates = df[df['is_duplicate']]
    non_duplicates = df[~df['is_duplicate']]
    
    most_complete_duplicates = duplicates.groupby(group_cols).apply(fill_from_group).reset_index(drop=True)
    
    filtered_df = pd.concat([non_duplicates, most_complete_duplicates], ignore_index=True)
    
    sorted_filtered_df = filtered_df.sort_values(by=['address', 'price', 'area'])
    
    return sorted_filtered_df

In [108]:
df_filtered_sorted_apartments = remove_duplicates(df_apartments)
df_filtered_sorted_apartments.drop('is_duplicate', axis=1, inplace=True)
df_filtered_sorted_apartments

C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\3201072169.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['is_duplicate'] = df.groupby(group_cols, group_keys=False).apply(check_similarity)
C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\3201072169.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_complete_duplicates = duplicates.groupby(group_cols).apply(fill_from_group).reset_index(drop=

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
15829,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność
15428,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN
15952,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN
13953,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność
10200,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15818,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN
16378,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN
10899,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność
11087,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",2722500.0,121.00,22500.00,2,4,4,2024.0,False,pierwotny,pełna własność


## Adresy

In [109]:
import re

# Regex pattern adjusted to exclude capturing prefixes directly
pattern = r'(ul\.|Aleja|aleja|pl\.|al\.)\s*([^,\d]+[\d]*\b)'

def update_address(row):
    if pd.isna(row['name/title']):
        return row['address']

    match = re.search(pattern, row['name/title'])
    if match:
        street_name = match.group(2).strip()
        if pd.isna(row['address']):
            updated_address = street_name
        else:
            if street_name.lower() not in row['address'].lower():
                updated_address = f"{street_name}, {row['address']}"
            else:
                updated_address = row['address']
    else:
        updated_address = row['address']

    # if pd.notna(updated_address) and 'kraków' not in updated_address.lower():
    #     updated_address += ", Kraków, małopolskie"
    
    return updated_address

In [110]:
df_filtered_sorted_apartments['address'] = df_filtered_sorted_apartments.apply(update_address, axis=1)
df_filtered_sorted_apartments['address'] = df_filtered_sorted_apartments['address'].str.replace(r'^ul\.\s+', '', regex=True)
df_filtered_sorted_apartments['address'] = df_filtered_sorted_apartments['address'].str.replace(r',\s*ul\.\s[^,\d]+,\s', ', ', regex=True)
df_filtered_sorted_apartments

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
15829,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność
15428,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN
15952,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN
13953,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność
10200,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15818,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN
16378,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN
10899,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność
11087,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",2722500.0,121.00,22500.00,2,4,4,2024.0,False,pierwotny,pełna własność


## Geolokacja

In [111]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="AGH_Student_Data_Exploration_Project")


def geocode(address):
    timeout = 10
    attempts = 0
    while attempts < 3:
        try:
            location = geolocator.geocode(address, timeout=timeout)
            if location:
                print(f"{location.address}, {location.latitude}, {location.longitude}")
                return pd.Series([location.address, location.latitude, location.longitude])
            else:
                return pd.Series([None, None, None])
        except Exception as e:
            print(f"Retry {attempts + 1}: Error during geocoding for address '{address}' - {str(e)}")
            attempts += 1
            timeout += 5
            time.sleep(2)
    return pd.Series([None, None, None])

In [165]:
unique_addresses = set(df_filtered_sorted_apartments['address'].unique())
len(unique_addresses)

3835

In [166]:
address_list = list(unique_addresses)

location_df = pd.DataFrame({
    'address': address_list,
    'manipulated address': address_list
})

In [114]:
import os

def geocode_and_save(temp_df, csv_path="datasets/geocoded_addresses.csv"):
    if 'location' not in temp_df:
        temp_df['location'] = pd.NA
    if 'latitude' not in temp_df:
        temp_df['latitude'] = pd.NA
    if 'longitude' not in temp_df:
        temp_df['longitude'] = pd.NA

    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path)
        existing_df = existing_df.drop(columns='manipulated address', errors='ignore')

        columns_to_drop = ['location', 'latitude', 'longitude']
        temp_df = temp_df.drop(columns=columns_to_drop, errors='ignore')

        merged_df = pd.merge(temp_df, existing_df, on='address', how='left')

        for col in columns_to_drop:
            if col + '_existing' in merged_df.columns:
                merged_df[col] = merged_df[col + '_existing'].combine_first(merged_df[col])
                merged_df.drop(col + '_existing', axis=1, inplace=True)

        needs_geocoding = merged_df['location'].isna()
        print(f"Rows needing geocoding: {needs_geocoding.sum()}")

        if needs_geocoding.any():
            geocoded_data = merged_df.loc[needs_geocoding, 'manipulated address'].apply(geocode)
            merged_df.loc[needs_geocoding, ['location', 'latitude', 'longitude']] = geocoded_data.values
        
        temp_df = merged_df
    else:
        geocoded_data = temp_df['manipulated address'].apply(geocode)
        temp_df[['location', 'latitude', 'longitude']] = geocoded_data.values

    temp_df.to_csv(csv_path, index=False)
    print(f"Data saved to {csv_path}")
    return temp_df


In [209]:
geocoded_file = 'datasets/geocoded_addresses.csv'
df_geocoded = pd.read_csv(geocoded_file)
df_unique = df_geocoded.drop_duplicates()
df_unique.to_csv(geocoded_file, index=False)

In [210]:
df2 = pd.read_csv('datasets/geocoded_from_xlsx.csv')
merged_df = pd.merge(df_geocoded, df2, on='address', how='left', suffixes=('', '_from_df2'))
df_geocoded.update(merged_df.filter(like='_from_df2').rename(columns=lambda x: x.replace('_from_df2', '')))
columns_to_drop = [col for col in df_geocoded.columns if '_from_df2' in col]
df_geocoded.drop(columns=columns_to_drop, axis=1, inplace=True)
df_geocoded.to_csv(geocoded_file, index=False)


In [167]:
location_df = geocode_and_save(location_df)

Rows needing geocoding: 0
Data saved to datasets/geocoded_addresses.csv


In [158]:
location_df[location_df['location'].isna()]

,address,manipulated address,location,latitude,longitude
1153,"Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...","Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...",None,None,None
2894,"Krzemienicka Prokocim z ogrodem, Kosocice, Swo...","Krzemienicka Prokocim z ogrodem, Kosocice, Swo...",None,None,None


In [117]:
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df['manipulated address'].str.replace(r',.*?, Kraków, małopolskie', ', Kraków, małopolskie', regex=True)

In [120]:
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df['manipulated address'].str.replace(r'^(ul\. |ul\.|os\. |U |Ok\. |abp\. |al\. |mrj\.|Św\. |Aleja gen\. |Św\. |prof\. |gen\. )', '', regex=True)

In [122]:
phrases_to_remove = [
    "okolice", "Wykończone inwestycyjne mieszkanie", "garsoniera wypoczynkowa",
    "⭐️ obok tramwaj", "EKOPARK", r"\| brak pcc \|", "wykończone bez PCC",
    "przy UP", "/bez pośredników", "- dwupoziomowe mieszkanie w centrum",
    " ENG", r"\(Os\.Piastow", " z ogródkiem", " - Bezpośrednio", " - stan deweloperski",
    r"\/ nowy apartament\/", " - Wykończone", " Tbs"
    ]

# Create a regex pattern string to match any of the phrases
phrases_pattern = r'|'.join([re.escape(phrase) for phrase in phrases_to_remove])

# Apply regex only where location is NaN
location_df.loc[location_df['location'].isna(), 'manipulated address'] = \
    location_df.loc[location_df['location'].isna(), 'manipulated address'].str.replace(phrases_pattern, '', regex=True)


In [124]:
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df[location_df['location'].isna()]['manipulated address'].str.replace(r'^-\s*,', ',', regex=True)
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df[location_df['location'].isna()]['manipulated address'].str.replace(r'^iej,', 'a,', regex=True)
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df[location_df['location'].isna()]['manipulated address'].str.replace(r'^ej,', 'a,', regex=True)

In [126]:
existing_df = pd.read_csv("geocoded_addresses.csv")
condition = existing_df['location'] == "Kraków, województwo małopolskie, Polska"
existing_df.loc[condition, ['location', 'latitude', 'longitude']] = None
existing_df.to_csv("geocoded_addresses.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'geocoded_addresses.csv'

In [127]:
location_df[location_df['location'] == "Kraków, województwo małopolskie, Polska"]

,address,manipulated address,location,latitude,longitude
68013,"64108, Kraków, małopolskie","64108, Kraków, małopolskie","Kraków, województwo małopolskie, Polska",50.061947,19.936856
68429,"Kraków, małopolskie","Kraków, małopolskie","Kraków, województwo małopolskie, Polska",50.046943,19.997153


In [160]:
# import openpyxl
na_location = location_df[location_df['location'].isna()]
na_location.to_excel("temp.xlsx", index=False)

In [162]:
geocode_and_save(pd.read_excel("temp.xlsx"), 'datasets/geocoded_from_xlsx.csv')

Rows needing geocoding: 2
Kuźnicy Kołłątajowskiej, Górka Narodowa, Prądnik Biały, Kraków, województwo małopolskie, 31-220, Polska, 50.0981141, 19.9599517
Krzemieniecka, Barycz, Kosocice, Swoszowice, Kraków, województwo małopolskie, 30-699, Polska, 49.9835607, 19.9961567
Data saved to datasets/geocoded_from_xlsx.csv


C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\3485834256.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Kuźnicy Kołłątajowskiej, Górka Narodowa, Prądnik Biały, Kraków, województwo małopolskie, 31-220, Polska', 'Krzemieniecka, Barycz, Kosocice, Swoszowice, Kraków, województwo małopolskie, 30-699, Polska']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged_df.loc[needs_geocoding, ['location', 'latitude', 'longitude']] = geocoded_data.values


,address,manipulated address,location,latitude,longitude
0,"Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...","Kuźnicy Kołłątajowskiej, Górka Narodowa, Prądn...","Kuźnicy Kołłątajowskiej, Górka Narodowa, Prądn...",50.098114,19.959952
1,"Krzemienicka Prokocim z ogrodem, Kosocice, Swo...","Krzemieniecka, Kosocice, Swoszowice, Kraków, m...","Krzemieniecka, Barycz, Kosocice, Swoszowice, K...",49.983561,19.996157


In [168]:
na_location[na_location['location'].isna()]

,address,manipulated address,location,latitude,longitude
1153,"Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...","Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...",None,None,None
2894,"Krzemienicka Prokocim z ogrodem, Kosocice, Swo...","Krzemienicka Prokocim z ogrodem, Kosocice, Swo...",None,None,None


In [169]:
merged_df = pd.merge(
    df_filtered_sorted_apartments, 
    location_df[['address', 'location', 'latitude', 'longitude']], 
    on='address', 
    how='left'
)

needs_update = merged_df['location'].isna()
merged_df.loc[needs_update, 'location'] = merged_df.loc[needs_update, 'location']
merged_df.loc[needs_update, 'latitude'] = merged_df.loc[needs_update, 'latitude']
merged_df.loc[needs_update, 'longitude'] = merged_df.loc[needs_update, 'longitude']

df_filtered_sorted_apartments = merged_df


In [170]:
df_filtered_sorted_apartments = df_filtered_sorted_apartments[df_filtered_sorted_apartments['location'] != "Kraków, województwo małopolskie, Polska"]

## Obliczanie odległości od centrum

In [171]:
# https://en.wikipedia.org/wiki/Haversine_formula
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Promień Ziemi w km
    r = 6371
    
    return c * r

In [172]:
# Srodek sukiennic
city_center_lat = 50.061853457016106
city_center_lon = 19.937306011671154

In [173]:
df_filtered_sorted_apartments['distance'] = df_filtered_sorted_apartments.apply(
    lambda row: haversine(city_center_lat, city_center_lon, row['latitude'], row['longitude']),
    axis=1
)
df_filtered_sorted_apartments

C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\2115724949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_sorted_apartments['distance'] = df_filtered_sorted_apartments.apply(


,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,location,latitude,longitude,distance
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17985,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
17986,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
17987,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
17988,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",2722500.0,121.00,22500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326


In [174]:
# Wyciąganie dzielnic
df_filtered_sorted_apartments['district'] = df_filtered_sorted_apartments['location'].str.extract(r'(?:^|, )([^,]+), Kraków')
df_filtered_sorted_apartments

C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\2136594925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_sorted_apartments['district'] = df_filtered_sorted_apartments['location'].str.extract(r'(?:^|, )([^,]+), Kraków')


,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,location,latitude,longitude,distance,district
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728,Mistrzejowice
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17985,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17986,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17987,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17988,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",2722500.0,121.00,22500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki


In [176]:
print(df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'].isna()].shape[0])
df_filtered_sorted_apartments = df_filtered_sorted_apartments.dropna(subset=['district'])

0


,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,location,latitude,longitude,distance,district
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728,Mistrzejowice
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17985,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17986,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17987,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17988,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",2722500.0,121.00,22500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki


In [223]:
temp_df = geocode('Zgodna, 32-031 Kraków, małopolskie')
temp_df

Zgodna, Knapówka, Libertów, gmina Mogilany, powiat krakowski, województwo małopolskie, 30-444, Polska, 49.9782828, 19.895051


0    Zgodna, Knapówka, Libertów, gmina Mogilany, po...
1                                            49.978283
2                                            19.895051
dtype: object

In [177]:
df_filtered_sorted_apartments.isnull().sum()

url                                       0
name/title                                0
address                                   0
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building       0
no of rooms                               0
year of construction                   1895
parking space                             0
market                                    0
form of ownership                      2381
location                                  0
latitude                                  0
longitude                                 0
distance                                  0
district                                  0
dtype: int64

In [178]:
nan_row_count = df_filtered_sorted_apartments.isna().any(axis=1).sum()
df_filtered_sorted_apartments.shape[0] - nan_row_count

14600

## Uzupełnienie NaNów i eksport zbioru uwspólnionego

In [179]:
df_filtered_sorted_apartments['year of construction'] = df_filtered_sorted_apartments['year of construction'].fillna(new_mean_year)
df_filtered_sorted_apartments['form of ownership'] = df_filtered_sorted_apartments['form of ownership'].fillna('pełna własność')
df_filtered_sorted_apartments.to_csv('datasets/apartments_collective.csv')
df_filtered_sorted_apartments

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,location,latitude,longitude,distance,district
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728,Mistrzejowice
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,2012.0,False,wtórny,pełna własność,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,pełna własność,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17985,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17986,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17987,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
17988,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",2722500.0,121.00,22500.00,2,4,4,2024.0,False,pierwotny,pełna własność,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki


## Usuwanie duplikatów, geolokacja, uzupełnienie NaNów i eksport zbioru nieruchomosci-online

In [180]:
df_filtered_sorted_nieruchomosci = remove_duplicates(df_nieruchomosci)
df_filtered_sorted_nieruchomosci.drop('is_duplicate', axis=1, inplace=True)
df_filtered_sorted_nieruchomosci

C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\3201072169.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['is_duplicate'] = df.groupby(group_cols, group_keys=False).apply(check_similarity)
C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\3201072169.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_complete_duplicates = duplicates.groupby(group_cols).apply(fill_from_group).reset_index(drop=

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details
6867,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,w garażu podziemnym
6466,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,parking publiczny / na ulicy
6990,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,parking publiczny / na ulicy
4990,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,garaż wolnostojący
1238,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,parking publiczny / na ulicy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8085,https://krakow.nieruchomosci-online.pl/mieszka...,"Kawalerka, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",532000.0,24.40,21803.28,1,2,1,1900.0,True,wtórny,NaN,tak
6856,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,tak
7416,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,parking strzeżony w pobliżu
1936,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,parking publiczny / na ulicy


In [195]:
merged_df = pd.merge(
    df_filtered_sorted_nieruchomosci,
    location_df_nieruchomosci[['address', 'location', 'latitude', 'longitude']],
    on='address',
    how='left',
    suffixes=('', '_from_location')
)


# Check and update the original columns conditionally
if 'location_from_location' in merged_df.columns:
    merged_df['location'] = merged_df['location_from_location'].combine_first(merged_df['location'])
    merged_df['latitude'] = merged_df['latitude_from_location'].combine_first(merged_df['latitude'])
    merged_df['longitude'] = merged_df['longitude_from_location'].combine_first(merged_df['longitude'])

    # Drop the _from_location columns if they exist
    cols_to_drop = ['location_from_location', 'latitude_from_location', 'longitude_from_location']
    cols_to_drop = [col for col in cols_to_drop if col in merged_df.columns]
    merged_df.drop(cols_to_drop, axis=1, inplace=True)
else:
    needs_update = merged_df['location'].isna()
    merged_df.loc[needs_update, 'location'] = merged_df.loc[needs_update, 'location']
    merged_df.loc[needs_update, 'latitude'] = merged_df.loc[needs_update, 'latitude']
    merged_df.loc[needs_update, 'longitude'] = merged_df.loc[needs_update, 'longitude']

# Update the original DataFrame with the merged data
df_filtered_sorted_nieruchomosci = merged_df


df_filtered_sorted_nieruchomosci[['address', 'location', 'latitude', 'longitude']].head()

,address,location,latitude,longitude
0,"103171, Mistrzejowice, Kraków, małopolskie","Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603
1,"2 Pułku Lotniczego, Kraków, małopolskie","2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307
2,"2 Pułku Lotniczego, Kraków, małopolskie","2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307
3,"28 Lipca 1943, Wola Justowska, Kraków, małopol...","28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002
4,"28 Lipca 1943, Wola Justowska, Kraków, małopol...","28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002


In [196]:
df_filtered_sorted_nieruchomosci[df_filtered_sorted_nieruchomosci['location'].isna()]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details,location,latitude,longitude


In [185]:
unique_addresses = set(df_filtered_sorted_nieruchomosci[df_filtered_sorted_nieruchomosci['location'].isna()]['address'].unique())
address_list = list(unique_addresses)

location_df_nieruchomosci = pd.DataFrame({
    'address': address_list,
    'manipulated address': address_list
})  

location_df_nieruchomosci.to_excel("temp.xlsx", index=False)
location_df_nieruchomosci

,address,manipulated address
0,"Bronowice Nowe, Kraków, małopolskie","Bronowice Nowe, Kraków, małopolskie"
1,"Łobzów, Kraków, małopolskie","Łobzów, Kraków, małopolskie"
2,"ul. Żeglarska 7, Zarzecze 34-326, Kraków, mało...","ul. Żeglarska 7, Zarzecze 34-326, Kraków, mało..."
3,"Armii Kraków, Jugowice, Kraków, małopolskie","Armii Kraków, Jugowice, Kraków, małopolskie"
4,"Kazimierza Siemianowicza, Płaszów, Kraków, mał...","Kazimierza Siemianowicza, Płaszów, Kraków, mał..."
5,"Gancarska, Stare Miasto, Kraków, małopolskie","Gancarska, Stare Miasto, Kraków, małopolskie"
6,"Armii ""Kraków"", Podgórze, Kraków, małopolskie","Armii ""Kraków"", Podgórze, Kraków, małopolskie"
7,"Lichecka, Łagiewniki, Kraków, małopolskie","Lichecka, Łagiewniki, Kraków, małopolskie"
8,"Armii Kraków, Swoszowice, Kraków, małopolskie","Armii Kraków, Swoszowice, Kraków, małopolskie"
9,"okolica ul. Mogilskiej, Grzegórzki, Kraków, ma...","okolica ul. Mogilskiej, Grzegórzki, Kraków, ma..."


In [194]:
location_df_nieruchomosci=geocode_and_save(pd.read_excel('temp.xlsx'), 'datasets/geocoded_from_xlsx.csv')
location_df_nieruchomosci

Rows needing geocoding: 0
Data saved to datasets/geocoded_from_xlsx.csv


,address,manipulated address,location,latitude,longitude
0,"Bronowice Nowe, Kraków, małopolskie","Bronowice, Kraków, małopolskie","Bronowice, Kraków, województwo małopolskie, Po...",50.083325,19.869784
1,"Łobzów, Kraków, małopolskie","Łobzów, małopolskie","Łobzów, gmina Wolbrom, powiat olkuski, wojewód...",50.405208,19.768476
2,"ul. Żeglarska 7, Zarzecze 34-326, Kraków, mało...","Zarzecze 34-326, małopolskie","Zarzecze, Frydrychowice, gmina Wieprz, powiat ...",49.891667,19.406389
3,"Armii Kraków, Jugowice, Kraków, małopolskie","Armii Kraków, Jugowice","Armii ""Kraków"", Jugowice, Swoszowice, Kraków, ...",50.003059,19.926099
4,"Kazimierza Siemianowicza, Płaszów, Kraków, mał...","Siemienowicza, Płaszów, Kraków, małopolskie","Bagry - ul. Lipska/Siemienowicza, Płaszów, Pod...",50.038014,19.995590
5,"Gancarska, Stare Miasto, Kraków, małopolskie","Garncarska, Stare Miasto, Kraków, małopolskie","Garncarska, Piasek, Stare Miasto, Kraków, woje...",50.062641,19.927712
6,"Armii ""Kraków"", Podgórze, Kraków, małopolskie","Armii Kraków, Kraków, małopolskie","Armii ""Kraków"", Jugowice, Swoszowice, Kraków, ...",50.003059,19.926099
7,"Lichecka, Łagiewniki, Kraków, małopolskie","Lechicka, Łagiewniki, Kraków, małopolskie","Lechicka, Łagiewniki, Łagiewniki-Borek Fałęcki...",50.024124,19.936961
8,"Armii Kraków, Swoszowice, Kraków, małopolskie","Armii Kraków, Kraków, małopolskie","Armii ""Kraków"", Jugowice, Swoszowice, Kraków, ...",50.003059,19.926099
9,"okolica ul. Mogilskiej, Grzegórzki, Kraków, ma...","Mogilska, Grzegórzki, Kraków, małopolskie","Mogilska, Osiedle Oficerskie, Grzegórzki, Krak...",50.066234,19.961421


In [197]:
df_filtered_sorted_nieruchomosci['distance'] = df_filtered_sorted_nieruchomosci.apply(
    lambda row: haversine(city_center_lat, city_center_lon, row['latitude'], row['longitude']),
    axis=1
)
df_filtered_sorted_nieruchomosci

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details,location,latitude,longitude,distance
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,w garażu podziemnym,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,garaż wolnostojący,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,parking publiczny / na ulicy,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,https://krakow.nieruchomosci-online.pl/mieszka...,"Kawalerka, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",532000.0,24.40,21803.28,1,2,1,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
8085,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
8086,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,parking strzeżony w pobliżu,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
8087,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,parking publiczny / na ulicy,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326


In [198]:
# Wyciąganie dzielnic
df_filtered_sorted_nieruchomosci['district'] = df_filtered_sorted_nieruchomosci['location'].str.extract(r'(?:^|, )([^,]+), Kraków')
df_filtered_sorted_nieruchomosci

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details,location,latitude,longitude,distance,district
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,w garażu podziemnym,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728,Mistrzejowice
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,garaż wolnostojący,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,parking publiczny / na ulicy,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,https://krakow.nieruchomosci-online.pl/mieszka...,"Kawalerka, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",532000.0,24.40,21803.28,1,2,1,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8085,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8086,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,parking strzeżony w pobliżu,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8087,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,parking publiczny / na ulicy,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki


In [199]:
print(df_filtered_sorted_nieruchomosci[df_filtered_sorted_nieruchomosci['district'].isna()].shape[0])
df_filtered_sorted_nieruchomosci = df_filtered_sorted_nieruchomosci.dropna(subset=['district'])

43


In [200]:
# Uzupełnianie
df_filtered_sorted_nieruchomosci['year of construction'] = df_filtered_sorted_nieruchomosci['year of construction'].fillna(new_mean_year)
df_filtered_sorted_nieruchomosci['form of ownership'] = df_filtered_sorted_nieruchomosci['form of ownership'].fillna('pełna własność')
df_filtered_sorted_nieruchomosci.to_csv('datasets/apartments_nieruchomosci.csv')
df_filtered_sorted_nieruchomosci

C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\1360996366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_sorted_nieruchomosci['year of construction'] = df_filtered_sorted_nieruchomosci['year of construction'].fillna(new_mean_year)
C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\1360996366.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_sorted_nieruchomosci['form of ownership'] = df_filtered_sorted_nieruchomosci['form of ownership'].fillna('pełna własność')


,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details,location,latitude,longitude,distance,district
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,w garażu podziemnym,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728,Mistrzejowice
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,2012.0,False,wtórny,pełna własność,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,pełna własność,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,garaż wolnostojący,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,parking publiczny / na ulicy,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,https://krakow.nieruchomosci-online.pl/mieszka...,"Kawalerka, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",532000.0,24.40,21803.28,1,2,1,1900.0,True,wtórny,pełna własność,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8085,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,pełna własność,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8086,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,pełna własność,parking strzeżony w pobliżu,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8087,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,parking publiczny / na ulicy,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki


In [201]:
df_filtered_sorted_nieruchomosci.isnull().sum()

url                                    0
name/title                             0
address                                0
price                                  0
area                                   0
price-per-area                         0
floor/store                            0
no of floors/stores in the building    0
no of rooms                            0
year of construction                   0
parking space                          0
market                                 0
form of ownership                      0
parking space details                  0
location                               0
latitude                               0
longitude                              0
distance                               0
district                               0
dtype: int64

## Usuwanie duplikatów, geolokacja, uzupełnienie NaNów i eksport zbioru otodom

In [202]:
df_filtered_sorted_otodom = remove_duplicates(df_otodom)
df_filtered_sorted_otodom.drop('is_duplicate', axis=1, inplace=True)
df_filtered_sorted_otodom

C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\3201072169.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['is_duplicate'] = df.groupby(group_cols, group_keys=False).apply(check_similarity)
C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\3201072169.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_complete_duplicates = duplicates.groupby(group_cols).apply(fill_from_group).reset_index(drop=

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
4504,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.3,12703.0,1,5,2,2023.0,...,pierwotny,pełna własność,NaN,NaN,miejskie,biuro nieruchomości,True,balkon,brak informacji,brak informacji
1144,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.0,17636.0,4,7,3,2023.0,...,wtórny,pełna własność,NaN,450.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
8962,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.8,16984.0,4,4,2,2019.0,...,wtórny,pełna własność,do zamieszkania,425.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
2387,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.0,13310.0,2,4,2,1964.0,...,wtórny,NaN,NaN,500.0,miejskie,biuro nieruchomości,False,balkon,blok,brak informacji
3990,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.1,14449.0,3,3,2,NaN,...,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,False,balkon,blok,brak informacji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.0,11039.0,4,6,3,2002.0,...,wtórny,pełna własność,NaN,800.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
7133,https://www.otodom.pl/pl/oferta/pradnik-bialy-...,Prądnik Biały - Ul. Reja,"Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,71.0,15493.0,0,6,3,2005.0,...,wtórny,NaN,do zamieszkania,NaN,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
7239,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.0,15068.0,0,6,3,2005.0,...,wtórny,pełna własność,NaN,700.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
388,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.5,16490.0,5,8,3,2001.0,...,wtórny,pełna własność,do zamieszkania,940.0,miejskie,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji


In [203]:
df_filtered_sorted_otodom['address'] = df_filtered_sorted_otodom.apply(update_address, axis=1)
df_filtered_sorted_otodom['address'] = df_filtered_sorted_otodom['address'].str.replace(r'^ul\.\s+', '', regex=True)
df_filtered_sorted_otodom['address'] = df_filtered_sorted_otodom['address'].str.replace(r',\s*ul\.\s[^,\d]+,\s', ', ', regex=True)
df_filtered_sorted_otodom

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
4504,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.3,12703.0,1,5,2,2023.0,...,pierwotny,pełna własność,NaN,NaN,miejskie,biuro nieruchomości,True,balkon,brak informacji,brak informacji
1144,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.0,17636.0,4,7,3,2023.0,...,wtórny,pełna własność,NaN,450.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
8962,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.8,16984.0,4,4,2,2019.0,...,wtórny,pełna własność,do zamieszkania,425.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
2387,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.0,13310.0,2,4,2,1964.0,...,wtórny,NaN,NaN,500.0,miejskie,biuro nieruchomości,False,balkon,blok,brak informacji
3990,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.1,14449.0,3,3,2,NaN,...,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,False,balkon,blok,brak informacji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.0,11039.0,4,6,3,2002.0,...,wtórny,pełna własność,NaN,800.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
7133,https://www.otodom.pl/pl/oferta/pradnik-bialy-...,Prądnik Biały - Ul. Reja,"Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,71.0,15493.0,0,6,3,2005.0,...,wtórny,NaN,do zamieszkania,NaN,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
7239,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.0,15068.0,0,6,3,2005.0,...,wtórny,pełna własność,NaN,700.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
388,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.5,16490.0,5,8,3,2001.0,...,wtórny,pełna własność,do zamieszkania,940.0,miejskie,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji


In [204]:
merged_df = pd.merge(
    df_filtered_sorted_otodom, 
    location_df[['address', 'location', 'latitude', 'longitude']], 
    on='address', 
    how='left'
)

needs_update = merged_df['location'].isna()

merged_df.loc[needs_update, 'location'] = merged_df.loc[needs_update, 'location']
merged_df.loc[needs_update, 'latitude'] = merged_df.loc[needs_update, 'latitude']
merged_df.loc[needs_update, 'longitude'] = merged_df.loc[needs_update, 'longitude']


df_filtered_sorted_otodom = merged_df

In [205]:
df_filtered_sorted_otodom[df_filtered_sorted_otodom['location'].isna()]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,rent,heating,advertiser type,elevator,outdoor area,building type,building material,location,latitude,longitude
3922,https://www.otodom.pl/pl/oferta/mieszkanie-ul-...,Mieszkanie ul. Bajeczna - wysoki standard,"Bajeczna - wysoki standard, Dąbie, Grzegórzki,...",1189000.0,56.67,20981.0,2,5,3,2023.0,...,650.0,NaN,prywatny,True,balkon,blok,brak informacji,NaN,NaN,NaN
7193,https://www.otodom.pl/pl/oferta/29-metrow-przy...,29 metrów przy ul. Pękowickiej - balkon i komórka,"Pękowickiej - balkon i komórka, Tonie, Prądnik...",492999.0,29.00,17000.0,3,3,1,2019.0,...,310.0,miejskie,biuro nieruchomości,False,None,blok,brak informacji,NaN,NaN,NaN
9856,https://www.otodom.pl/pl/oferta/mieszkanie-ul-...,Mieszkanie ul. Lea po generalnym remoncie,"Lea po generalnym remoncie, Łobzów, Krowodrza,...",995000.0,48.00,20729.0,7,10,3,1990.0,...,NaN,miejskie,prywatny,True,balkon,blok,brak informacji,NaN,NaN,NaN


In [207]:
unique_addresses = set(df_filtered_sorted_otodom[df_filtered_sorted_otodom['location'].isna()]['address'].unique())
address_list = list(unique_addresses)

location_df_otodom = pd.DataFrame({
    'address': address_list,
    'manipulated address': address_list
})

location_df_otodom.to_excel("temp.xlsx", index=False)

In [208]:
location_df_otodom=geocode_and_save(pd.read_excel('temp.xlsx'), 'datasets/geocoded_from_xlsx.csv')
location_df_otodom

Rows needing geocoding: 3
Juliusza Lea, Łobzów, Krowodrza, Kraków, województwo małopolskie, 30-070, Polska, 50.0723293, 19.9050999
Pękowicka, Tonie, Prądnik Biały, Kraków, gmina Zielonki, powiat krakowski, województwo małopolskie, 31-264, Polska, 50.1103712, 19.9119922
Bajeczna, Dąbie, Grzegórzki, Kraków, województwo małopolskie, 31-566, Polska, 50.0574115, 19.9766744
Data saved to datasets/geocoded_from_xlsx.csv


,address,manipulated address,location,latitude,longitude
0,"Lea po generalnym remoncie, Łobzów, Krowodrza,...","Lea, Łobzów, Krowodrza, Kraków, małopolskie","Juliusza Lea, Łobzów, Krowodrza, Kraków, wojew...",50.072329,19.905100
1,"Pękowickiej - balkon i komórka, Tonie, Prądnik...","Pękowicka, Tonie, Prądnik Biały, Kraków, małop...","Pękowicka, Tonie, Prądnik Biały, Kraków, gmina...",50.110371,19.911992
2,"Bajeczna - wysoki standard, Dąbie, Grzegórzki,...","Bajeczna, Dąbie, Grzegórzki, Kraków, małopolskie","Bajeczna, Dąbie, Grzegórzki, Kraków, województ...",50.057412,19.976674


In [211]:
merged_df = pd.merge(
    df_filtered_sorted_otodom,
    location_df_otodom[['address', 'location', 'latitude', 'longitude']],
    on='address',
    how='left',
    suffixes=('', '_from_location')
)
if 'location_from_location' in merged_df.columns:
    merged_df['location'] = merged_df['location_from_location'].combine_first(merged_df['location'])
    merged_df['latitude'] = merged_df['latitude_from_location'].combine_first(merged_df['latitude'])
    merged_df['longitude'] = merged_df['longitude_from_location'].combine_first(merged_df['longitude'])
    merged_df.drop(['location_from_location', 'latitude_from_location', 'longitude_from_location'], axis=1, inplace=True)

df_filtered_sorted_otodom.update(merged_df)

In [212]:
df_filtered_sorted_otodom['distance'] = df_filtered_sorted_otodom.apply(
    lambda row: haversine(city_center_lat, city_center_lon, row['latitude'], row['longitude']),
    axis=1
)
df_filtered_sorted_otodom

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,heating,advertiser type,elevator,outdoor area,building type,building material,location,latitude,longitude,distance
0,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.3,12703.0,1,5,2,2023.0,...,miejskie,biuro nieruchomości,True,balkon,brak informacji,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.103296,19.963896,4.983478
1,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.0,17636.0,4,7,3,2023.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Adama Vetulaniego, Prądnik Biały, Kraków, woje...",50.095951,19.924784,3.895335
2,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.8,16984.0,4,4,2,2019.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Agatowa, Osiedle Kolejowe, Złocień, Bieżanów-P...",50.022046,20.046131,8.943634
3,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.0,13310.0,2,4,2,1964.0,...,miejskie,biuro nieruchomości,False,balkon,blok,brak informacji,"Osiedle Albertyńskie, Bieńczyce, Kraków, wojew...",50.081431,20.017628,6.131841
4,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.1,14449.0,3,3,2,NaN,...,gazowe,biuro nieruchomości,False,balkon,blok,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.099768,19.962839,4.592695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.0,11039.0,4,6,3,2002.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967
9899,https://www.otodom.pl/pl/oferta/pradnik-bialy-...,Prądnik Biały - Ul. Reja,"Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,71.0,15493.0,0,6,3,2005.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967
9900,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.0,15068.0,0,6,3,2005.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967
9901,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.5,16490.0,5,8,3,2001.0,...,miejskie,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967


In [213]:
# Wyciąganie dzielnic
df_filtered_sorted_otodom['district'] = df_filtered_sorted_otodom['location'].str.extract(r'(?:^|, )([^,]+), Kraków')
df_filtered_sorted_otodom

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,advertiser type,elevator,outdoor area,building type,building material,location,latitude,longitude,distance,district
0,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.3,12703.0,1,5,2,2023.0,...,biuro nieruchomości,True,balkon,brak informacji,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.103296,19.963896,4.983478,Prądnik Biały
1,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.0,17636.0,4,7,3,2023.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Adama Vetulaniego, Prądnik Biały, Kraków, woje...",50.095951,19.924784,3.895335,Prądnik Biały
2,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.8,16984.0,4,4,2,2019.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Agatowa, Osiedle Kolejowe, Złocień, Bieżanów-P...",50.022046,20.046131,8.943634,Bieżanów-Prokocim
3,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.0,13310.0,2,4,2,1964.0,...,biuro nieruchomości,False,balkon,blok,brak informacji,"Osiedle Albertyńskie, Bieńczyce, Kraków, wojew...",50.081431,20.017628,6.131841,Bieńczyce
4,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.1,14449.0,3,3,2,NaN,...,biuro nieruchomości,False,balkon,blok,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.099768,19.962839,4.592695,Prądnik Biały
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.0,11039.0,4,6,3,2002.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały
9899,https://www.otodom.pl/pl/oferta/pradnik-bialy-...,Prądnik Biały - Ul. Reja,"Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,71.0,15493.0,0,6,3,2005.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały
9900,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.0,15068.0,0,6,3,2005.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały
9901,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.5,16490.0,5,8,3,2001.0,...,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały


In [214]:
print(df_filtered_sorted_otodom[df_filtered_sorted_otodom['district'].isna()].shape[0])
# df_filtered_sorted_otodom[df_filtered_sorted_otodom['district'].isna()]
df_filtered_sorted_otodom = df_filtered_sorted_otodom.dropna(subset=['district'])

3


In [215]:
# Tu zaczyna się uzupełnianie brakujących danych
df_filtered_sorted_otodom['year of construction'] = df_filtered_sorted_otodom['year of construction'].fillna(new_mean_year)
df_filtered_sorted_otodom['rent'] = df_filtered_sorted_otodom['rent'].fillna(new_mean_rent)
df_filtered_sorted_otodom['form of ownership'] = df_filtered_sorted_otodom['form of ownership'].fillna('pełna własność')
df_filtered_sorted_otodom['condition'] = df_filtered_sorted_otodom['condition'].fillna('brak informacji')
df_filtered_sorted_otodom['heating'] = df_filtered_sorted_otodom['heating'].fillna('brak informacji')
df_filtered_sorted_otodom.to_csv('datasets/apartments_otodom.csv')
df_filtered_sorted_otodom.isnull().sum()

C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\4294965649.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_sorted_otodom['year of construction'] = df_filtered_sorted_otodom['year of construction'].fillna(new_mean_year)
C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\4294965649.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_sorted_otodom['rent'] = df_filtered_sorted_otodom['rent'].fillna(new_mean_rent)
C:\Users\Piotr\AppData\Local\Temp\ipykernel_7468\4294965649.py:4: Settin

url                                    0
name/title                             0
address                                0
price                                  0
area                                   0
price-per-area                         0
floor/store                            0
no of floors/stores in the building    0
no of rooms                            0
year of construction                   0
parking space                          0
market                                 0
form of ownership                      0
condition                              0
rent                                   0
heating                                0
advertiser type                        0
elevator                               0
outdoor area                           0
building type                          0
building material                      0
location                               0
latitude                               0
longitude                              0
distance        